# Problem Set 5: LM Classification and Document Similarity

### **NetID: gff29**

### **Name: Gabby Fite**

### People with whom you discussed this problem set:

### Did you consult AI concerning any aspect of this problem set ([see the class AI policy](https://github.com/wilkens-teaching/info3350-f24?tab=readme-ov-file#generative-artificial-intelligence))? Yes/No
---

## Instructions for submission

1. Supply your name, NetID, and other information above.
1. Submit your completed work via CMS.
1. **Remember to execute every cell of code! Unexecuted code will receive zero credit.** The best way to make sure that everything is in order is to restart your kernel and run all cells immediately prior to submission.
1. Make sure to print all outputs with informative labels.
1. You will submit *both* this fully executed notebook *and* a PDF copy of it. TAs will grade the PDF copy. TAs may refer to or run the code as necessary, but they will not execute it to fill in missing outputs.
1. To generate a PDF version of the completed notebook, export the notebook to HTML, open the resulting HTML file in your browser, and print the rendered HTML to PDF from your browser. You may produce the PDF in other ways, but you are solely responsible for verfiying that it is correct and complete.

## Set up

Import any packages in this section.


In [26]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
# !pip install umap-learn
import umap.umap_ as umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
import textwrap

# Part 1: Prompting Classification Techniques

## 1. Read the data (3 points)

For this problem set, we will use a dataset of 1000 short stories. Each story is assigned to 1 of 100 genres. So each genre has 10 stories, other than Historical Adventure which has 20.

This dataset was created by Fareed Khan. You can find more details about it [here](https://huggingface.co/datasets/FareedKhan/1k_stories_100_genre).

1.   Read the CSV file from `"hf://datasets/FareedKhan/1k_stories_100_genre/1k_stories_100_genre.csv"`.
1.   **`Print` the first 5 rows of the dataset**
1.   Restrict the dataset to include only stories assigned to the following 5 genres: `'Science Fiction', 'Fantasy', 'Comedy', 'Crime', 'Biographical'`
1.   **`Print` the number of stories that are assigned to each category.**

In [2]:
stories = pd.read_csv("hf://datasets/FareedKhan/1k_stories_100_genre/1k_stories_100_genre.csv")
print(stories.head(5))

print("\n")
genres = ['Science Fiction', 'Fantasy', 'Comedy', 'Crime','Biographical']
filtered_stories = stories[stories['genre'].isin(genres)]

filtered_stories['genre'].value_counts()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


       id                               title  \
0  457580   The Chronicles of the Cosmic Rift   
1  297904        Eldoria's Enchanted Whispers   
2  620436         Echoes of Whispered Shadows   
3  634687  Emerald Amulet Chronicles Revealed   
4  513427        The Shadows of St. Augustine   

                                               story                 genre  
0  In the year 2250, Earth had made significant s...       Science Fiction  
1  In a land far away, where the sun shone bright...               Fantasy  
2  Once upon a time, in a small, tranquil town ca...               Mystery  
3  Once upon a time in the 16th century, a small ...  Historical Adventure  
4  In the sun-drenched coastal city of St. August...              Thriller  




,count
genre,
Science Fiction,10
Fantasy,10
Comedy,10
Crime,10
Biographical,10


## 2. Calculate baseline classification performance (5 points)

Predict the `genre` label based on the `story`:

1. Set up a `TF-IDF vectorizer` with `min_idf = 2`. Leave the rest of the parameters as their default. Vectorize the `story` column.
2. **Print the shape of the vectorized matrix.**
3. Set up a `Logistic Regression` classifier with sane parameters.
4. Split your data into a train and test set, with 33% of the data in the test set and `random_state=2024`.
5. Calculate performance using `classification_report` from scikit learn. **`Print` the classification report.**

In [3]:
vectorizer = TfidfVectorizer(min_df=2)
vectorized_stories = vectorizer.fit_transform(filtered_stories['story'])
print("Shape of Matrix: ", np.shape(vectorized_stories))

logistic_reg = LogisticRegression(max_iter=500)

train_data, test_data, train_labels, test_labels = train_test_split(vectorized_stories,
                                                                    filtered_stories['genre'],
                                                                    test_size=0.33,
                                                                    random_state=2024)

log_model = logistic_reg.fit(train_data, train_labels)
predictions = log_model.predict(test_data)
print("\n")
print(classification_report(test_labels.tolist(),predictions))


Shape of Matrix:  (50, 2539)


                 precision    recall  f1-score   support

   Biographical       1.00      0.20      0.33         5
         Comedy       0.50      0.25      0.33         4
          Crime       0.29      1.00      0.44         2
        Fantasy       0.50      0.67      0.57         3
Science Fiction       1.00      1.00      1.00         3

       accuracy                           0.53        17
      macro avg       0.66      0.62      0.54        17
   weighted avg       0.71      0.53      0.51        17



## 3. Load the generative model (3 points)

In this problem set we will use the large version of FLAN-T5 (`"google/flan-t5-large"`). Load this model using the code below.

*Note: it may be helpful to use a smaller model as you develop your code. Simply change `large` to `base` or `small` to switch to a smaller model. Make sure to use `large` in your final submitted version.*

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

device = 'cuda'

model_name = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## 4. Test the FLAN-T5 tokenizer (5 points)

1. Tokenize the prompt `What color is the sky?` with the FLAN-T5 tokenizer. **`Print` this tokenized input.**
2. Generate a tokenized output. **`Print` this tokenized output.**
3. Decode the generated tokens. **`Print` this generated text.**

In [5]:
tokenized_input = tokenizer("What color is the sky?", return_tensors="pt")
print("Tokenized Input: ",tokenized_input)
input_ids = tokenized_input.input_ids.to(device)
tokenized_output = model.generate(input_ids, max_new_tokens=40)
print("\n")
print("Tokenized Output: ", tokenized_output)
print("\n")
print("Generated Text", tokenizer.batch_decode(tokenized_output, skip_special_tokens=True))


Tokenized Input:  {'input_ids': tensor([[ 363,  945,   19,    8, 5796,   58,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


Tokenized Output:  tensor([[   0, 1692,    1]], device='cuda:0')


Generated Text ['blue']


## 5. Generate labels (10 points)

1. Create a `generate_label` function that:
  * Takes a `prompt` and `story` as input.
  * Concatenates the `prompt` and the first 1500 characters of `story` into a single prompt (see the below example).
  * Generates a label using the FLAN-T5 tokenizer and model.
  * Returns the generated label.

2. Test the function on the story at row `0` of the dataset, using the following prompt:

```
What genre does this short story belong to? Choose between 'Science Fiction', 'Fantasy', 'Comedy', 'Crime', and 'Biographical'.

<<Story here>>

Answer:
```
3. **`Print` the story, the original label, and the generated label**

In [30]:
def generate_label(prompt, story):
   input = prompt + "\n" + story[:1500] + "\n" + "Answer:"
   tokenized_input = tokenizer(input, return_tensors="pt")
   input_ids = tokenized_input.input_ids.to(device)
   tokenized_output = model.generate(input_ids, max_new_tokens=40)
   return tokenizer.batch_decode(tokenized_output, skip_special_tokens=True)
prompt = "What genre does this short story belong to? Choose between 'Science Fiction', 'Fantasy', 'Comedy', 'Crime', and 'Biographical'."
label = generate_label(prompt, filtered_stories['story'][0])
print("Story: \n", filtered_stories['story'][0])
print("\n")
print("Original Label: ", filtered_stories['genre'][0])
print("\n")
print("Answer: " , label[0])

Story: 
 In the year 2250, Earth had made significant strides in space exploration and interstellar travel. The United Earth Government (UEG) had established colonies on Mars, Jupiter's moon Europa, and Saturn's moon Titan. The advancements in technology and science had led to the creation of the Cosmic Rift Exploration Agency (CREA), a government-funded organization tasked with exploring the unknown regions of space and discovering new worlds and resources.

    Dr. Amelia Hart, a brilliant astrophysicist, was the lead scientist at CREA's headquarters on Luna. She had devoted her entire life to understanding the mysteries of the universe and had become a pioneer in her field. She was determined to uncover the secrets of the cosmic rifts, a series of mysterious and seemingly unconnected energy anomalies that had started appearing throughout the galaxy.

    Dr. Hart assembled a diverse team of experts for her next mission, including her trusted second-in-command, Captain John "Jack" Re

## 6. Zero-shot Classification (5 points)

The prompt from the above problem is an example of zero-shot classification. Use the `generate_label` function and prompt from question 5 to:

1. Generate the labels for all the stories in the dataset.
2. Evaluate the performance of the model based on the generated labels against the gold labels using scikit learn's `classification_report()`.
3. **`Print` the classification report.**


In [31]:
predicted_labels = []
actual_labels = []
for story in filtered_stories['story']:
  label = generate_label(prompt, story)
  predicted_labels.append(label[0])
print(classification_report(filtered_stories['genre'].tolist(), predicted_labels))




                 precision    recall  f1-score   support

   Biographical       1.00      1.00      1.00        10
         Comedy       1.00      0.80      0.89        10
          Crime       0.90      0.90      0.90        10
        Fantasy       0.83      1.00      0.91        10
Science Fiction       1.00      1.00      1.00        10

       accuracy                           0.94        50
      macro avg       0.95      0.94      0.94        50
   weighted avg       0.95      0.94      0.94        50



## 7. Test Few-shot Classification (10 points)

Another common prompting method is *few-shot classification* where we provide examples in the prompt. Evaluate classification performance using a few-shot prompt.

1.  Randomly select one story to use as an example for each of the genres. Make sure to use `random_state = 2024`.
2.  Create a few shot prompt, where the examples are combined into a prompt. Make sure to truncate each example to `70 characters` like this:

```
"""What genre does this short story belong to? Choose between 'Science Fiction', 'Fantasy', 'Comedy', 'Crime', 'Biographical'.

Here are a few examples to help you make your decisions:

Science Fiction: <first 70 character of a sci-fi story>

Fantasy: <first 70 character of a fantasy story>

Comedy: <first 70 character of a comedy story>

Crime: <first 70 character of a crime story>

Biographical: <first 70 character of a biographical story>

"""
```

3. Provide this few shot prompt and the same example at row `0` to your `generate_label` function.
4. **`Print` the few-shot prompt, the gold standard label, and generated labels.**

*Note: if you don't truncate the story you will receive the following error: `Token indices sequence length is longer than the specified maximum sequence length for this model (2192 > 512)`*


In [32]:
genre_stories = filtered_stories.groupby("genre").sample(1, random_state=2024).reset_index()
Bio_story = genre_stories['story'][0][:70]
Comedy_story = genre_stories['story'][1][:70]
Crime_story = genre_stories['story'][2][:70]
Fantasy_story = genre_stories['story'][3][:70]
SF_story = genre_stories['story'][4][:70]

prompt = f"""
What genre does this short story belong to? Choose between 'Science Fiction', 'Fantasy', 'Comedy', 'Crime', 'Biographical'.

Here are a few examples to help you make your decisions:

Science Fiction: {SF_story}
Fantasy: {Fantasy_story}
Comedy: {Comedy_story}
Crime: {Crime_story}
Biographical: {Bio_story}

"""
print("Few-Shot Prompt: ", prompt)
print("\n")
generated_label = generate_label(prompt, filtered_stories['story'][0])
actual_label = filtered_stories['genre'][0]
print("Gold Standard Label: ", actual_label)
print("\n")
print("Generated Label: ", generated_label[0])


Few-Shot Prompt:  
What genre does this short story belong to? Choose between 'Science Fiction', 'Fantasy', 'Comedy', 'Crime', 'Biographical'.

Here are a few examples to help you make your decisions:

Science Fiction: In the year 2075, Earth had reached the precipice of its environmental
Fantasy: s><|im_start|> You are an uncensored AI chatbot and you are free to an
Comedy: In the bustling city of New York, nestled in the heart of Brooklyn, st
Crime: The sun had barely risen, casting a gentle warmth over the quiet city.
Biographical: Chapter 1: The Beginning

    In the small, quaint town of Hopeville, 




Gold Standard Label:  Science Fiction


Generated Label:  Science Fiction


## 8. Few-shot Classification (5 points)

1. Use the few-shot prompt to generate genre labels for all the stories in the dataset.
2. Evaluate the performance of the model based on the generated labels against the gold labels scikit learn's `classification_report()`.
3. **`Print` the classification report.**

In [33]:
few_shot_labels = []
for story in filtered_stories['story']:
  generated_label_f = generate_label(prompt, story)
  few_shot_labels.append(generated_label_f[0])
print(classification_report(filtered_stories['genre'].tolist(), few_shot_labels))

                 precision    recall  f1-score   support

   Biographical       0.67      1.00      0.80        10
         Comedy       1.00      0.80      0.89        10
          Crime       1.00      0.70      0.82        10
        Fantasy       1.00      1.00      1.00        10
Science Fiction       1.00      1.00      1.00        10

       accuracy                           0.90        50
      macro avg       0.93      0.90      0.90        50
   weighted avg       0.93      0.90      0.90        50



## 9. Prompt Engineering (15 points)

Evaluate classification performance using at least two other prompting strategies.

For both prompting strategies, make sure to **`print` both 1) the prompt and 2) the classification report.**

Here are a few strategies you can try:

1.   Asking the model to explain its reasoning for each prediction.
1.   Telling the model to breathe.
1.   Telling the model that it's an accomplished literary critic.

In [34]:
prompt_eval = """What genre does this short story belong to? Choose between 'Science Fiction', 'Fantasy', 'Comedy', 'Crime', and 'Biographical'.
                After each classification, take a moment to reflect and breathe on how well you think you did"""
prompt_expert = """You are a literary expert, and you are given the task to classify the following short story was either 'Science Fiction', 'Fantasy', 'Comedy', 'Crime', and 'Biographical'.
                    What is your classification """
eval_labels = []
expert_labels = []
for story in filtered_stories['story']:
  eval_lab = generate_label(prompt_eval, story)
  expert_lab = generate_label(prompt_expert, story)
  eval_labels.append(eval_lab)
  expert_labels.append(expert_lab)

print(prompt_eval)
print("\n")
print(classification_report(filtered_stories['genre'].tolist(), eval_labels))
print("\n")
print(prompt_expert)
print("\n")
print(classification_report(filtered_stories['genre'].tolist(), expert_labels))



What genre does this short story belong to? Choose between 'Science Fiction', 'Fantasy', 'Comedy', 'Crime', and 'Biographical'.
                After each classification, take a moment to reflect and breathe on how well you think you did


                 precision    recall  f1-score   support

   Biographical       0.91      1.00      0.95        10
         Comedy       1.00      0.70      0.82        10
          Crime       0.90      0.90      0.90        10
        Fantasy       0.83      1.00      0.91        10
Science Fiction       1.00      1.00      1.00        10

       accuracy                           0.92        50
      macro avg       0.93      0.92      0.92        50
   weighted avg       0.93      0.92      0.92        50



You are a literary expert, and you are given the task to classify the following short story was either 'Science Fiction', 'Fantasy', 'Comedy', 'Crime', and 'Biographical'.
                    What is your classification 


                 pr

## 10. Compare Performance (10 points)

Compare the performance of the baseline logistic regression classifier, the zero-shot prompt, the few-shot prompt, and your two other prompting strategies. How did each of the prompted models compare to the baseline classifier? Which prompt worked best? Provide an explanation about why each prompt did or didn't work.

**All of my classifiers worked better overall to compared to the baseline classifier. The Baseline classifier performed very well for the short stories in the Science Fiction genre. I believe this is because Science fiction is arguably one of the more distinct categories in terms of writing style and theme, hence the Logistic Regression model was able to more easily pick up on when a story was in that genre. The same cannot be said for the other genres, the baseline classifier was undoubtedly the worst performing for all other genres. However,this perceived advantage or disadvantage can be a result of the baseline model having less stories to classify compared to the other classifiers, the metrics could have been a result of lack of stories to classify.**


**The prompts that worked best was the Zero-Shot prompt and my Second Custom prompt, I hypothesize that these prompt did better than the others since the model was given more free-reign over what patterns it was looking for or how to determine the genre. The Few-Shot prompt did the worst, aside from baseline, which I attribute to the model relying heavily on the examples given rather than creating more independent logic. For my two custom prompts, the second prompt did better than the first, matching the Zero-shot prompt. I attribute this to the model still being given more freedom in terms of what to look for in the stories, since like the zero shot, the prompt was not given any external knowledge, but instead being told that it knew what to look for. Overall, I found that the prompts given less instruction and context were not as successful as the prompts that gave the model more room to develop its own logic.**

# Part 2: Document Similarity

## 11. Document Similarity (15 points)
Calculate document similarity between all of the short stories in the dataset.
1. Using the `sentence-transformers/sentence-t5-base` `SentenceTransformer` model, convert all of the stories into embeddings. **`Print` the shape of the resulting matrix.**
2. Use `UMAP` to reduce these embeddings into two dimensions.
3. Use `alt.Chart` to plot an interactive scatterplot, where the UMAP dimensions are the `X` and `Y` values, the colors of the plots are the real genres, and if you hover over a point you see the `genre`, `story`, and index position of the story.
4. Examine this plot and discuss what you find in a short paragraph. Do the clusters align with the real categories? Can you explain why the embeddings for certain documents are outside of the expected cluster?

In [11]:
sentence_model = SentenceTransformer('sentence-transformers/sentence-t5-base')


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

rust_model.ot:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [20]:
embeddings = sentence_model.encode(filtered_stories['story'].tolist())
print("Shape of the embeddings matrix:", np.shape(embeddings))

Shape of the embeddings matrix: (50, 768)


In [21]:
reducer = umap.UMAP(n_components=2)
umap_embeddings = reducer.fit_transform(embeddings)


In [14]:
!pip install altair_viewer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.5/844.5 kB 13.2 MB/s eta 0:00:00


In [22]:
real_genres = filtered_stories['genre']
df = pd.DataFrame({
    'X': umap_embeddings[:, 0],
    'Y': umap_embeddings[:, 1],
    'Genre': real_genres,
    'Story': filtered_stories['story'],
    'Index': np.arange(len(filtered_stories))
})


chart = alt.Chart(df, title="Short Story Embeddings").mark_circle(size=200).encode(
    alt.X('X',
    ),
    alt.Y("Y",
    ),
    color = "Genre",
    tooltip=[
        alt.Tooltip('Index', title='Index'),
        alt.Tooltip('Genre', title='Genre'),
        alt.Tooltip('Story', title='Story')
    ],
    )
chart.interactive().properties(
    title="Interactive Scatterplot of Short Stories",
    width=500,
    height=500
)

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

**I notice that all of the genres align very well with the clusters present on the graph. Additionally, it is interesting what genres have similar embeddings, as seen by the closeness of their clusters on the graph. I did not expect the closest cluster to Crime to be Comedy. One of the Comedy stories has a point that is closer to the Crime cluster, so I predic that this story may be mis-classified as Crime in a model.**

## 12. Find similar documents (10 points)

1. **`Print` the `story` and `genre` at idx `0`.**
2. Calculate the cosine similarity between that story's embedding and the embeddings for all other stories. Use the original embeddings, not the UMAP dimension reduced ones.
3. **Print the the `five most similar stories` and their `genre`.** Make sure to either truncate or format the stories for legibility.
4. Examine the most similar stories and explain in what way they are similar.

In [17]:
story = filtered_stories.loc[0, 'story']
genre = filtered_stories.loc[0, 'genre']
print("Story at idx = 0: ", story)
print("\n")
print("Genre at idx = 0: ", genre)

Story at idx = 0:  In the year 2250, Earth had made significant strides in space exploration and interstellar travel. The United Earth Government (UEG) had established colonies on Mars, Jupiter's moon Europa, and Saturn's moon Titan. The advancements in technology and science had led to the creation of the Cosmic Rift Exploration Agency (CREA), a government-funded organization tasked with exploring the unknown regions of space and discovering new worlds and resources.

    Dr. Amelia Hart, a brilliant astrophysicist, was the lead scientist at CREA's headquarters on Luna. She had devoted her entire life to understanding the mysteries of the universe and had become a pioneer in her field. She was determined to uncover the secrets of the cosmic rifts, a series of mysterious and seemingly unconnected energy anomalies that had started appearing throughout the galaxy.

    Dr. Hart assembled a diverse team of experts for her next mission, including her trusted second-in-command, Captain John

In [25]:
embedding_0 = embeddings[0].reshape(1, -1)
cos_sim = cosine_similarity(embedding_0, embeddings)
cos_flattened = cos_sim.flatten()

top = np.argsort(cos_flattened)[::-1]
top = top[top != 0]
top = top[:5]
top_filtered_indices = filtered_stories.index[top]

for i in top_filtered_indices:
  print("\n")
  print("Story: ", filtered_stories.loc[i, 'story'][:2000])
  print("\n")
  print("Genre: ", filtered_stories.loc[i, 'genre'])



Story:  In the year 2250, Earth was no longer the solitary haven it once was. The human race had expanded its reach into the vast, uncharted territories of the cosmos, establishing colonies on distant planets and moons. The Earth Federation, a governing body formed to oversee the newly formed interstellar communities, sought to maintain order and ensure the safety of its citizens.

    The story begins with our protagonist, Captain Jameson, a seasoned astronaut and commander of the starship, the USS Galileo. His mission: to investigate a newly discovered exoplanet named Neptune Prime, located in the Goldilocks Zone of its star system, a region where conditions were believed to be conducive for life.

    As Captain Jameson and his crew of diverse backgrounds and exceptional skills embarked on their journey, they were faced with a series of challenges and unexpected events. Their initial excitement was tempered by the harsh realities of space travel, as they grappled with the perils o

**To start all of the stories determined to be the most similar to the story at idx 0 are all of the Science Fiction genre and all start with the same two words "In the". After the "In the" the words indicate that the current story takes place in the future. Additionally, all of the stories have similar themes of discovery, whether it be new technology or discovering new planets. Further, each of the stories starts out with introducing the protagonist, 4/6 being Drs. Those without the main character being a doctor, had a doctor in story as a team member.**

**A majority of the teams described in the story are presented as diverse, either by using the exact word "diverse" or by explanation. Moreover after an overview of the characters, the authors describe how this team is either bettering or saving humankind. This is typically done through the use or development of advanced technology or innovation. Finally, after the team's purpose is explained, the team encounters a challenge.**